In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange

import torch
from torch import nn
import torch.nn.functional as F
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms

from tqdm import tqdm

from torchinfo import summary

In [2]:
# First define the encoder, the encoder is a bert like transformer encoder. It inserts a [CLS] token at the beginning, followed by joint position, followed by k action sequences.

class ActionEncoder(nn.Module):
    def __init__(self, num_actions, joint_dim, lantent_dim = 512):
        super().__init__()
        self.input_dim = joint_dim + num_actions * joint_dim
        self.lantent_dim = lantent_dim
        self.cls_token = nn.Parameter(
            torch.rand(
                1, lantent_dim
            )
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=lantent_dim, nhead=2
            ),  # joint pos and action seq
            num_layers=2,
        )
        
        self.joints_mlp = nn.Linear(joint_dim, lantent_dim)
        self.actions_mlp = nn.Linear(joint_dim, lantent_dim)

    def forward(self, joints, actions):
        joints = self.joints_mlp(joints)
        actions = self.actions_mlp(actions)
        # print(f'joints: {joints.shape}, actions: {actions.shape}, cls: {self.cls_token.shape}')
        X = torch.cat((self.cls_token, joints, actions), dim=0)
        # print(f'X: {X.shape}')
        X = self.transformer_encoder(X)
        return X[0].reshape(-1, self.lantent_dim)


model = ActionEncoder(num_actions=4, joint_dim=14, lantent_dim=128)
summary(model=model, input_size=[(1, 14), (4, 14)])

/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Layer (type:depth-idx)                        Output Shape              Param #
ActionEncoder                                 [1, 128]                  128
├─Linear: 1-1                                 [1, 128]                  1,920
├─Linear: 1-2                                 [4, 128]                  1,920
├─TransformerEncoder: 1-3                     [6, 128]                  --
│    └─ModuleList: 2-1                        --                        --
│    │    └─TransformerEncoderLayer: 3-1      [6, 128]                  593,024
│    │    └─TransformerEncoderLayer: 3-2      [6, 128]                  593,024
Total params: 1,190,016
Trainable params: 1,190,016
Non-trainable params: 0
Total mult-adds (M): 6.33
Input size (MB): 0.00
Forward/backward pass size (MB): 0.24
Params size (MB): 4.23
Estimated Total Size (MB): 4.47

In [3]:
class ActionDecoder(nn.Module):
    def __init__(self, lantent_dim, num_actions):
        super().__init__()
        self.latent_dim = lantent_dim
        self.num_actions = num_actions
        
        self.resnet = torchvision.models.resnet18(pretrained=True)
        self.resnet.avgpool = nn.Identity()
        self.resnet.fc = nn.Identity()
        
        self.joints_mlp = nn.Linear(14, lantent_dim)

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(d_model=lantent_dim, nhead=4),
            num_layers=2,
        )

        def get_sinusoidal_embeddings(num_embeddings, embedding_dim):
            position = torch.arange(
                0, num_embeddings, dtype=torch.float).unsqueeze(1)
            div_term = torch.exp(torch.arange(
                0, embedding_dim, 2).float() * (-np.log(10000.0) / embedding_dim))
            embeddings = torch.zeros(num_embeddings, embedding_dim)
            embeddings[:, 0::2] = torch.sin(position * div_term)
            embeddings[:, 1::2] = torch.cos(position * div_term)
            return embeddings

        self.position_embeddings = nn.Parameter(
            get_sinusoidal_embeddings(num_actions, lantent_dim), requires_grad=False)

        self.transformer_decoder = nn.TransformerDecoder(
            decoder_layer=nn.TransformerDecoderLayer(d_model=lantent_dim, nhead=4),
            num_layers=2,
        )
        self.mlp = nn.Linear(lantent_dim, 14)

    def forward(self, images, z, joint_pos):
        # images shape: [4, 3, 480, 640]
        features = self.resnet(images)
        features = rearrange(features, 'b (d c) -> (b d) c', c=self.latent_dim)
        # features shape: [1200, 512]
        
        joint_pos = self.joints_mlp(joint_pos)

        # Concatenate z and joint_pos with features
        combined_input = torch.cat((features, z, joint_pos), dim=0)
        # combined_input shape: [1202, 512]

        # Encode combined input using transformer encoder
        encoded_features = self.transformer_encoder(combined_input)
        # encoded_features shape: [1202, 512]

        # Decode using transformer decoder
        tgt = self.position_embeddings
        memory = encoded_features
        decoded_output = self.transformer_decoder(tgt, memory)
        # Map to final output dimension using MLP
        output = self.mlp(decoded_output)

        return output

In [4]:
transformer_encoder = ActionDecoder(lantent_dim=128, num_actions=4)
summary(model=transformer_encoder, input_size=[(4, 3, 480, 640), (1, 128), (1, 14)])

/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                        Output Shape              Param #
ActionDecoder                                 [4, 14]                   512
├─ResNet: 1-1                                 [4, 153600]               --
│    └─Conv2d: 2-1                            [4, 64, 240, 320]         9,408
│    └─BatchNorm2d: 2-2                       [4, 64, 240, 320]         128
│    └─ReLU: 2-3                              [4, 64, 240, 320]         --
│    └─MaxPool2d: 2-4                         [4, 64, 120, 160]         --
│    └─Sequential: 2-5                        [4, 64, 120, 160]         --
│    │    └─BasicBlock: 3-1                   [4, 64, 120, 160]         73,984
│    │    └─BasicBlock: 3-2                   [4, 64, 120, 160]         73,984
│    └─Sequential: 2-6                        [4, 128, 60, 80]          --
│    │    └─BasicBlock: 3-3                   [4, 128, 60, 80]          230,144
│    │    └─BasicBlock: 3-4                   [4, 128, 60, 80]          295,4

In [5]:
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()
        self.encoder = ActionEncoder(num_actions=4, joint_dim=14, lantent_dim=128)
        self.mu_phi = nn.Linear(128, 128)
        self.logvar = nn.Linear(128, 128)
        # output action seq directly
        self.decoder = ActionDecoder(lantent_dim=128, num_actions=4)

    def encode(self, joints, actions):
        z = self.encoder(joints, actions)
        return self.mu_phi(z), self.logvar(z)

    def reparametrize(self, mu_phi, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std).detach().to(device)
        return mu_phi + eps * std

    def decode(self, images, z, joint_pos):
        return self.decoder(images, z, joint_pos)

    def forward(self, current_joints, future_actions, images):
        mu_phi, logvar = self.encode(current_joints, future_actions)
        z = self.reparametrize(mu_phi, logvar)
        return self.decode(images, z, current_joints), mu_phi, logvar

    def loss_fn(self, predict, target, mu_phi, logvar):
        # KL divergence: Sum over latent dimensions, average over batch
        kl_divergence = -0.5 * \
            torch.sum(1 + logvar - mu_phi.pow(2) - logvar.exp(), dim=1).mean()
        # Reconstruction loss: Mean squared error
        # Sum over features, average over batch
        reconstruction = F.mse_loss(predict, target, reduction='mean')

        # Total loss
        return kl_divergence + reconstruction

In [6]:
model = CVAE()
summary(model=model, input_size=[(1, 14), (4, 14), (4, 3, 480, 640)])

Layer (type:depth-idx)                             Output Shape              Param #
CVAE                                               [4, 14]                   --
├─ActionEncoder: 1-1                               [1, 128]                  128
│    └─Linear: 2-1                                 [1, 128]                  1,920
│    └─Linear: 2-2                                 [4, 128]                  1,920
│    └─TransformerEncoder: 2-3                     [6, 128]                  --
│    │    └─ModuleList: 3-1                        --                        1,186,048
├─Linear: 1-2                                      [1, 128]                  16,512
├─Linear: 1-3                                      [1, 128]                  16,512
├─ActionDecoder: 1-4                               [4, 14]                   512
│    └─ResNet: 2-4                                 [4, 153600]               --
│    │    └─Conv2d: 3-2                            [4, 64, 240, 320]         9,408
│    │   